In [3]:
import pandas as pd

In [4]:
class Util():
    TRAIN_SESSION = "./data/train_session.csv"
    RANDOM_SUBMISSION = "./data/random_submission.csv"
    TEST_TRACKING = "./data/test_tracking.csv"
    TRAIN_TRACKING = "./data/train_tracking.csv"
    PRODUCT_CATEGORY = "./data/productid_category.csv"
    
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

In [5]:
print("LOADING TRAIN")
df_train = load_data(Util.TRAIN_SESSION)
print(df_train.head())
print(df_train.describe())

print("LOADING TRAIN")
df_train_tracking = load_data(Util.TRAIN_TRACKING)
print(df_train_tracking.head())
print(df_train_tracking.describe())

print("LOADING TEST")
df_test = load_data(Util.TEST_TRACKING)
print(df_test.head())
print(df_test.describe())

print("LOADING SUBMISSION")
df_submission = load_data(Util.RANDOM_SUBMISSION)
print(df_submission.head())
print(df_submission.describe())

print("LOADING PRODUCT")
df_product = load_data(Util.PRODUCT_CATEGORY)
print(df_product.head())
print(df_product.describe())

LOADING TRAIN
                                                 sid  target
0  U6clt0UYaQB7vJQFmSWYymsAfwmT9SMhrm2oXQ8TC5M8mf...   False
1  0qRbLkxeOjeEo6CtxtB6VI8FNPt4Vl8niySzZLqSWiQOka...   False
2  cnCRGomfPqEWuhMZkA0RQfvTQiwVoyHXUqvqnPCutVouFO...   False
3  zBwrofN0r2ps9u/UCUS134SiZIqB+UgIEr0MZGHzksqR4f...   False
4  Kbr60r9eaX/ENOJ81R1YBxfwrEohoxyQI9Ma0fODsS+/XN...   False
                                                      sid  target
count                                              133123  133123
unique                                             133123       2
top     lnn8btjMVDcH4d6NqsGYpKZ0hFmP4RrnJljKAKUdl4eEKa...   False
freq                                                    1  120643
LOADING TRAIN
                                                 sid      type query  \
0  xvmQh3WmJGKaplwGgHf4o1MNKg2/6IpkbKNh4nnwZibi3f...  CAROUSEL   NaN   
1  xvmQh3WmJGKaplwGgHf4o1MNKg2/6IpkbKNh4nnwZibi3f...  CAROUSEL   NaN   
2  xvmQh3WmJGKaplwGgHf4o1MNKg2/6IpkbKNh4nnwZibi3f...  CAROUS

FileNotFoundError: File b'./data/productid_category.csv' does not exist

In [21]:
balancing = df_train[(df_train==1)].count()/df_train.count()
print(balancing)

sid       0.000000
target    0.093748
dtype: float64


In [45]:
def boolToInt(b):
    if b==True:
        return 1
    else:
        return 0

def isBasket(event_strign):
    return  "ADD_TO_BASKET" in event_strign
    
    
df_baseline = df_test[["sid", "type"]]
df_baseline["target"] = df_baseline["type"].apply(isBasket).apply(boolToInt)
df_baseline = df_baseline[["sid", "target"]]
df_baseline = df_baseline.groupby("sid").sum()
df_baseline["target"] = df_baseline["target"].clip(0.1,1)

C:\Users\lucas\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [46]:
import io, math, requests

def submit_prediction(df, sep=',', comment='', **kwargs):
    TOKEN='d74622182150aab24c843d04cea81eb8f68a58467ba4ef8925d176cbeb06c879df07a4c8ce5aef542db892ac99dde01d787d9bc13666ecbd458344e96b51ff6a'
    URL='https://qscore.datasciencegame.com/api/submissions'
    buffer = io.StringIO()
    df.to_csv(buffer, sep=sep, **kwargs)
    buffer.seek(0)
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': buffer},data={'comment':comment})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)

submit_prediction(df_baseline, sep=',', index=True, comment='dummy baseline')